<a href="https://colab.research.google.com/github/its-relative/IntelliSQR/blob/main/intelliSQR_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting the Drive to load the PDF

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installing the Libraries

In [2]:
!pip install transformers accelerate pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.3 MB/s eta 0:00:00


# Loading the Model

In [3]:
# ---------- CELL 1: Load Mistral model ----------
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

print("🚀 Loading Mistral model (this may take several minutes)...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("✅ Model loaded successfully!")


🚀 Loading Mistral model (this may take several minutes)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

✅ Model loaded successfully!


# Table Extraction

In [4]:
# ---------- CELL 2: PDF → PAN-Name Extraction ----------
import pdfplumber
import re
import pandas as pd

PDF_PATH = r"/content/drive/MyDrive/IntelliSSR/PDF for Python LLM (1).pdf"   # path to PDF file
# PDF_PATH = "PDF for Python LLM (1).pdf"
OUTPUT_CSV = "pan_relations_filtered.csv"

# ---------- STEP 1: Extract text (pages 1–3) ----------
def extract_text_from_pdf(pdf_path, start_page=0, end_page=3):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            if i < start_page:
                continue
            if i >= end_page:
                break
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


# ---------- STEP 2: Extract PAN–Name pairs ----------
def extract_name_pan_pairs(text):
    pan_pattern = r"[A-Z]{5}[0-9]{4}[A-Z]{1}"
    lines = text.split("\n")
    pairs = []
    for line in lines:
        line = line.strip()
        match = re.search(pan_pattern, line)
        if match:
            pan = match.group()
            name = line.replace(pan, "").strip().lstrip("0123456789.- ").strip()
            if len(name) > 2:
                pairs.append((name, pan))
    return pairs


# ---------- STEP 3: Use already loaded LLM ----------
def clean_with_llm(name, pan):
    prompt = f"""
    Format this data into a clean triple:
    Name: {name}
    PAN: {pan}
    Output format: <PAN>, PAN_Of, <Name>.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# ---------- STEP 4: Run pipeline ----------
print(" Extracting text from pages 1–3...")
text = extract_text_from_pdf(PDF_PATH, start_page=0, end_page=3)

print(" Extracting PAN–Name pairs...")
pairs = extract_name_pan_pairs(text)
print(f" Found {len(pairs)} valid entries between pages 1–3.")

print(" Cleaning with LLM...")
results = []
for i, (name, pan) in enumerate(pairs, 1):
    print(f" [{i}/{len(pairs)}] Processing: {name[:40]}...")
    cleaned = clean_with_llm(name, pan)
    results.append(cleaned)

# ---------- STEP 5: Save to CSV ----------
# data = []
# for r in results:
#     match = re.match(r"([A-Z0-9]+),?\s*PAN_Of,?\s*(.+)", r)
#     if match:
#         data.append({"PAN": match.group(1), "Relation": "PAN_Of", "Entity": match.group(2).strip()})

# df = pd.DataFrame(data)
# df.to_csv(OUTPUT_CSV, index=False)
# print(f"\n Extraction complete! Saved as {OUTPUT_CSV}")


📄 Extracting text from pages 1–3...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔍 Extracting PAN–Name pairs...
✅ Found 70 valid entries between pages 1–3.
🧠 Cleaning with LLM...
🧠 [1/70] Processing: MAHESHWARI FINANCIAL SERVICES PVT. LTD....


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 [2/70] Processing: AUTOLITE AGENCIES PVT. LTD....


...

🧠 [68/70] Processing: AAMIR MEMON...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 [69/70] Processing: MADHUR BUILDCON PVT. LTD....


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 [70/70] Processing: SURBHI INFRAPROJECT PVT. LTD....

✅ Extraction complete! Saved as pan_relations_filtered.csv


# Saving to CSV

In [23]:
# ---------- STEP 5: Save to CSV (robust parsing) ----------
data = []

for r in results:
    # Clean up line breaks and extra spaces
    text = r.replace("\n", " ").strip()

    # Try matching patterns like:
    # (AAPPG1345M, PAN_Of, "SANJEEV GOEL")
    # or Triple: AAPPG1345M, PAN_Of, SANJEEV GOEL
    match = re.search(
        r"([A-Z]{5}[0-9]{4}[A-Z])[^A-Z]*PAN_Of[^A-Z]*['\"]?([A-Z\s\.]+)['\"]?",
        text
    )

    if match:
        pan = match.group(1).strip()
        name = match.group(2).strip()
        data.append({
            "PAN": pan,
            "Relation": "PAN_Of",
            "Entity": name
        })
    else:
        # if no match found, just log it
        print(f"⚠️ Could not parse: {text[:80]}")

# Save DataFrame
df = pd.DataFrame(data)
df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Extraction complete! Saved as {OUTPUT_CSV}")


⚠️ Could not parse: Format this data into a clean triple:     Name: TOOR FINANCE COMPANY LTD.     PA
⚠️ Could not parse: Format this data into a clean triple:     Name: E TRICKS ENTERPRISES PVT. LTD.  

✅ Extraction complete! Saved as pan_relations_filtered.csv


# Checking CSV content

In [24]:
data = pd.read_csv(r"/content/pan_relations_filtered.csv")

In [26]:
df = data.copy()

In [27]:
df

,PAN,Relation,Entity
0,AAACM9185B,PAN_Of,MAHESHWARI FINANCIAL SERVICES PVT. LTD.
1,AAECA1487G,PAN_Of,AUTOLITE AGENCIES PVT. LTD.
2,AAACS3356A,PAN_Of,STELLAR CAPITAL SERVICES LTD.
3,ACRPR2362R,PAN_Of,PREMLAL ROY
4,AAVFA6230F,PAN_Of,ARIES COMMERCIALS.
...,...,...,...
63,AUOPB2860R,PAN_Of,ASHOK SNEHA BOHRA
64,AWJPG9626R,PAN_Of,RAHUL GOEL
65,BSFPM3219K,PAN_Of,AAMIR MEMON
66,AAFCM9969G,PAN_Of,MADHUR BUILDCON PVT. LTD.


```
🧾 Conclusion

In this notebook, we extracted PAN–Entity relationships from SEBI PDF documents using the **Mistral-7B-Instruct-v0.2** model.
Text was parsed from pages 1–3, cleaned, and formatted into triples.

Out of 70 potential entries, the model correctly extracted and parsed 68, resulting in a success rate of **97.14%**. The extracted data is saved in the `pan_relations_filtered.csv` file.
```